<a href="https://colab.research.google.com/github/femketenharkel/Predicting_Ratings/blob/main/models/clustering/DBSCAN_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Link with drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Loading in the dataframe
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")

In [13]:
pd.set_option('display.max_columns', None)
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Age,Release_year,Total_ratings_per_user,Female,Male,Academic/educator,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Favourite_Action,Favourite_Adventure,Favourite_Animation,Favourite_Children's,Favourite_Comedy,Favourite_Crime,Favourite_Documentary,Favourite_Drama,Favourite_Fantasy,Favourite_Film-Noir,Favourite_Horror,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western
0,1645,485,3,2000,11,23,12,1,1993,522,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,3112,357,4,2000,9,20,20,1,1994,213,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2,2077,2792,2,2000,11,19,19,1,1982,684,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,868,2279,4,2000,11,26,23,5,1998,75,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,2773,1103,4,2000,11,1,10,3,1955,83,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


## Normalize the data

In [14]:
df_scaled = df_final.copy()

In [15]:
df_scaled.columns

Index(['UserID', 'MovieID', 'Rating', 'Year', 'Month', 'Day', 'Hour', 'Age',
       'Release_year', 'Total_ratings_per_user', 'Female', 'Male',
       'Academic/educator', 'Artist', 'Clerical/admin', 'College/grad student',
       'Customer service', 'Doctor/health care', 'Executive/managerial',
       'Farmer', 'Homemaker', 'K-12 student', 'Lawyer',
       'Other or not specified', 'Programmer', 'Retired', 'Sales/marketing',
       'Scientist', 'Self-employed', 'Technician/engineer',
       'Tradesman/craftsman', 'Unemployed', 'Writer', 'Action', 'Adventure',
       'Animation', 'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'Favourite_Action',
       'Favourite_Adventure', 'Favourite_Animation', 'Favourite_Children's',
       'Favourite_Comedy', 'Favourite_Crime', 'Favourite_Documentary',
       'Favourite_Drama', 'Favourite_Fantasy', 'Favourite_Film-Noir

In [17]:
# Normalize columns for more accurate clustering

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Columns to normalize
columns_to_normalize = ['Year', 'Month', 'Day', 'Hour', 'Release_year', 'Total_ratings_per_user']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the specified columns
df_scaled[columns_to_normalize] = scaler.fit_transform(df_scaled[columns_to_normalize])


In [19]:
# One-hot encode age labels for clustering (otherwise it would falsely
# suggest that there is a numerical relationship)

df_scaled = pd.get_dummies(df_scaled, columns=['Age'], prefix='AgeGroup', prefix_sep='_')


In [23]:
# klopt niet, aantal clusters is ongeveer aantal instances -> nog aanpassen


import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

# Extract features for DBSCAN (excluding 'MovieID', 'Rating', and 'UserID')
features = df_scaled.drop(columns=['MovieID', 'Rating', 'UserID'])

# Apply DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='euclidean')
clusters = dbscan.fit_predict(features)

# Add cluster labels back to the DataFrame
df_scaled['Cluster'] = clusters


In [25]:
df_scaled['Cluster'].unique()

array([    0,     1,     2, ..., 49932, 49933, 49934])